In [2]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout,  Flatten
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.layers import Input 
from keras import backend as K
import  matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing import image as kimage

from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import preprocess_input
import numpy as np
import tensorflow as tf
from scipy import optimize

#from keras.preprocessing.image import img_to_array
from scipy.optimize import fmin_l_bfgs_b
import time

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# functions + gradient

## 标准用法: 以占位符实例化

In [61]:
inputs = K.placeholder(shape=(2, 2))
outputs = K.dot(inputs, inputs) 

func = K.function([inputs],[outputs]) # 实例化一个张量函数

val = np.random.random((2, 2))
func([val]) # 输入为numpy, 输出为numpy

[array([[0.17359288, 0.28688848],
        [0.26178834, 0.44428176]], dtype=float32)]

In [52]:
val = np.random.random((2, 2))
func([val]) # 结果是不一样的。

[array([[0.36415818, 0.5888605 ],
        [0.15553114, 0.5672088 ]], dtype=float32)]

## 以variable 实例化输入function

 这个也可以？ 比较怪

In [49]:
val2 = np.random.random((2, 2))
a = K.variable(value=val)
b = K.variable(value=val)
c = K.dot(a,b)

func = K.function([a,b],[c])  # 竟然可以用variable来示例化一个function
func([val2,val2])

[array([[0.44650674, 0.15564309],
        [0.9874884 , 0.93421614]], dtype=float32)]

In [50]:
val2 = np.random.random((2, 2))
func([val2,val2])  

[array([[0.7739289 , 0.40827876],
        [0.41796717, 0.76521915]], dtype=float32)]

## 计算梯度

In [79]:
inputs = K.placeholder(shape=(2, 2))
outputs = K.dot(inputs, inputs) 

grad = K.gradients(outputs, inputs) 
print(type(grad))

val = np.random.random((2, 2))

# 注：计算梯度出的梯度grad，但是不能单独调用 grad([x]),将会有问题，因为grad是个张量。 
# 将输入和输出的映射，定义为一个函数。然后输入才可以。
g_func = K.function([inputs],grad)
print(val)
print(g_func([val]))
print(K.function([inputs],[K.dot(inputs,inputs)])([val]))

<class 'list'>
[[0.04770285 0.67023043]
 [0.75408434 0.92022279]]
[array([[1.5197206, 2.4760942],
       [2.3083863, 3.2647603]], dtype=float32)]
[array([[0.5076859, 0.6487332],
       [0.7298975, 1.3522203]], dtype=float32)]


In [80]:
# 其中：推导x11的导数为：2*x11 + x21+ x12
0.04770285 * 2 + 0.67023043 + 0.75408434

1.5197204700000002